In [2]:
import os

In [3]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'a:\\mlflow\\Heart-Failure-Prediction-with-Machine-Learning-MLflow'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path 

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,    
        schema_file_path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root_dir])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config
        
    



In [10]:
from urllib import request
import zipfile
from mlProject.utils.common import get_size
from mlProject import logger

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{file_name} downloaded with following info: {headers}")
            logger.info(f"File size: {get_size(Path(self.config.local_data_file))}")

        else:
            logger.info(f"File {self.config.local_data_file} already exists.")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        


In [13]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    print(e)
    raise e

[2025-07-21 18:43:20,343 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-21 18:43:20,345 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-21 18:43:20,349 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-21 18:43:20,351 : INFO: created directory at: artifacts]
[2025-07-21 18:43:20,351 : INFO: created directory at: artifacts/data_ingestion]
[2025-07-21 18:43:20,351 : INFO: File artifacts\data_ingestion\data.zip already exists.]
